In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
from src.models.EMOS_global.EMOS_global_load_score import *
from src.models.EMOS_local.EMOS_local_load_score import *
from src.models.EMOS_local.EMOS_local_load_model import *
from src.models.EMOS_global.EMOS_global_load_model import *
from src.models.EMOS_local.EMOS_local_hyper_tune import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *
from src.models.EMOS_local.EMOS_local_train import *

<IPython.core.display.Javascript object>

### 1. Load Scores

In [7]:
# Load EMOS local scores
EMOS_local_t2m_scores = EMOS_local_load_score("t2m")
# EMOS_local_ws10_scores = EMOS_local_load_score("ws10")

# Load EMOS global scores
EMOS_global_t2m_scores = EMOS_global_load_score("t2m")
EMOS_global_ws10_scores = EMOS_global_load_score("ws10")

# Load CRPS scores
CRPS_global_t2m_scores = crps_load_lead_lat_lon("t2m")
CRPS_global_ws10_scores = crps_load_lead_lat_lon("ws10")

<IPython.core.display.Javascript object>

In [8]:
len(EMOS_local_t2m_scores)

0

<IPython.core.display.Javascript object>

In [14]:
EMOS_local_t2m_scores[0].shape

IndexError: list index out of range

<IPython.core.display.Javascript object>

In [15]:
EMOS_local_t2m_scores[0]

IndexError: list index out of range

<IPython.core.display.Javascript object>

### 3. Load Hyperparameter tuning:

In [4]:
EMOS_local_ws10_hyper_scores = EMOS_local_load_hyper_score("ws10")

<IPython.core.display.Javascript object>

In [5]:
EMOS_local_ws10_hyper_scores

[[(32, 30, 0.1, 'Adam'), 0.5986838517471765]]

<IPython.core.display.Javascript object>

### 3. Trying to load models:

In [ ]:
EMOS_local_t2m_0 = EMOS_local_load_model(2, 0)

In [24]:
lat = 2
lon = 2
lead_time = 0

<IPython.core.display.Javascript object>

In [28]:
test_var_denormed = ldpd.load_data_all_train_proc_denorm()[2]

<IPython.core.display.Javascript object>

In [29]:
X_test_var_denormed = test_var_denormed[
    list(test_var_denormed.data_vars.keys())[0]
].isel(lead_time=lead_time, lat=lat, lon=lon)
y_test_var_denormed = test_var_denormed[
    list(test_var_denormed.data_vars.keys())[1]
].isel(lead_time=lead_time, lat=lat, lon=lon)

<IPython.core.display.Javascript object>

In [30]:
EMOS_local_t2m_0[lat][lon].evaluate(
    [
        X_test_var_denormed.isel(mean_std=0).values.flatten(),
        X_test_var_denormed.isel(mean_std=1).values.flatten(),
    ],
    y_test_var_denormed.values.flatten(),
)

45/45 [==============================] - 0s 927us/step - loss: 402.3280


402.3279724121094

<IPython.core.display.Javascript object>